<a href="https://colab.research.google.com/github/Oreolorun/Customer-Churn-Banking-/blob/main/Building_Neural_Nets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#  Importing libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms, datasets

### Creating Torch Datasets

In [3]:
#  loading training data
train_ds = datasets.MNIST(root='data/', train=True, download=True,
                          transform=transforms.Compose([transforms.ToTensor()]))
#  loading test data
test_ds = datasets.MNIST(root='data/', train=False, download=True,
                          transform=transforms.Compose([transforms.ToTensor()]))

### Creating Splitting Function

In [12]:
#  creating split function
def split_train_ds(dataset, val_pct):
  #  size of dataset
  size_ds = len(dataset)
  #  desired size of validation set
  size_val = int(size_ds*val_pct)
  #  creating random permutations
  perm = np.random.permutation(size_ds)
  return perm[:size_val], perm[size_val:]

val_indices, train_indices = split_train_ds(train_ds, 0.2) 

### Creating Sampler and Loading Data

In [14]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

#  creating samplers
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

batch_size=10
#  creating loaders
train_dl = DataLoader(train_ds, sampler=train_sampler, batch_size=batch_size)
valid_dl = DataLoader(train_ds, sampler=val_sampler, batch_size=batch_size,)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

### Checking for Imbalance

In [30]:
#  checking for class imbalance
counter_dict = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0}
for img, label in train_dl:
  for lab in label:
    counter_dict[int(lab)]+=1

counter_dict

{0: 4721,
 1: 5367,
 2: 4768,
 3: 4935,
 4: 4662,
 5: 4338,
 6: 4709,
 7: 5012,
 8: 4726,
 9: 4762}

### Defining The Network

In [55]:
import torch.nn.functional as F
import torch.nn as nn

class NeuralNet(nn.Module):
  """This class models a neural network"""
  
  def __init__(self):
    """An initialisation method which 
    inherits from the parent class nn.Module.
    This is also where network layers are defined"""
    super().__init__()
    self.fc1 = nn.Linear(784,64)
    self.fc2 = nn.Linear(64,64)
    self.fc3 = nn.Linear(64,64)
    self.fc4 = nn.Linear(64,10)
  
  def forward(self, x):
    x = x.view(-1, 784)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return F.log_softmax(x, dim=1)



net = NeuralNet()
net

NeuralNet(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)

### Defining Accuracy function

In [135]:
#  Defining metric
def accuracy(validation_set):
  lbs = []
  preds = []
  with torch.no_grad():
    for im, lb in validation_set:
      for l in lb:
        lbs.append(l)
      for i in im:
        output = net(i)
        _, ind = torch.max(output, dim=1)
        preds.append(ind)
    k=0
    for j in range(len(lbs)):
      if preds[j]==lbs[j]:
        k+=1
  return round(k/12000, 3)

### Training Model

In [140]:
#  Training model
import torch.optim as optim

optimiser = optim.Adam(net.parameters(), lr=1e-5)

epochs = 10

for epoch in range(epochs):
  for data, labels in train_dl:
    #  zero gradients
    net.zero_grad()
    #  make predictions
    predictions = net(data)
    #  calculate loss
    loss = F.nll_loss(predictions, labels)
    #  compute gradients
    loss.backward()
    #  optimise gradients
    optimiser.step()
  #  print loss
  print(f'Epoch [{epoch+1}/{epochs}]\tLoss: {round(loss.item(), 6)}')

Epoch [1/10]	Loss: 2e-06
Epoch [2/10]	Loss: 0.000207
Epoch [3/10]	Loss: 0.000675
Epoch [4/10]	Loss: 1e-06
Epoch [5/10]	Loss: 3.1e-05
Epoch [6/10]	Loss: 0.00014
Epoch [7/10]	Loss: 0.001772
Epoch [8/10]	Loss: 0.00186
Epoch [9/10]	Loss: 6.3e-05
Epoch [10/10]	Loss: 7e-06


In [141]:
accuracy(test_dl)

0.813